# COMP47590: Advanced Machine Learning
# Assignment 1: Implementing Perceptrons

- Student 1 Name: Carl Fabian Winkler
- Student 1 Number: 20207528
- Student 2 Name: David Moreno Boras
- Student 2 Number: 21200646

## Import Packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import math
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import train_test_split

"""
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.utils.validation import check_X_y, check_array, check_is_fitted, check_random_state
from sklearn.utils.multiclass import unique_labels
from sklearn import preprocessing
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.utils import resample"""

'\nfrom sklearn.model_selection import GridSearchCV\nfrom sklearn.model_selection import cross_val_score\nfrom sklearn.model_selection import train_test_split\n\nfrom sklearn.utils.validation import check_X_y, check_array, check_is_fitted, check_random_state\nfrom sklearn.utils.multiclass import unique_labels\nfrom sklearn import preprocessing\nfrom sklearn import metrics\nfrom sklearn.model_selection import GridSearchCV\nfrom sklearn.model_selection import cross_val_score\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.utils import resample'

## Task 1: The Perceptron Classifier

Define the PerceptronClassifier class

In [140]:
class Layer():
    def __init__(self, n_in, n_out, activation = 'Sigmoid', init = 'Xavier'):
        self.activation = activation
        # XW + b = y ; We input more than one sample per pass...
        
        self.weights = np.zeros((n_in, n_out))
        self.biases = np.zeros((n_out))
        if init == 'Xavier':
            var = np.sqrt(6.0 / (n_in + n_out))
            for i in range(n_in):
                for j in range(n_out):
                      self.weights[i,j] = np.float32(np.random.uniform(-var, var))
        
        self.d_w = np.zeros(weights.shape)
        self.d_b = np.zeros(biases.shape)
        #print("Weights:", self.weights.shape)
        #print(self.weights) 
        
    def getWeights(self):
        return self.weights

    def forward(self, x):
        """print("X:", x.shape)
        print(x)
        print("Weights:", self.weights.shape)
        print(self.weights)"""
        z = x @ self.weights + self.biases
        #print("Weights:", self.weights.shape)
        if self.activation == 'Sigmoid':
            out = 1 / (1 + np.exp(-z))
        elif self.activation == 'ReLu':
            out = np.maximum(z, 0)
        elif self.activation == 'TanH':
            out = np.tanh(z)
        else:
            out = z
        
        self.cache = (x, z)
        
        return out    
    
    def backward(self, d_out):
        inputs, z = self.cache
        weight = self.weights
        bias = self.biases
        #print(self.weights)
        if self.activation == 'Sigmoid':
            d_act = d_out * (1 / (1 + np.exp(-z))) * (1 - 1 / (1 + np.exp(-z)))
        elif self.activation == 'ReLu':
            d_act = d_out * (z > 0)
            
        elif self.activation == 'TanH':
            d_act = d_out * np.square(z)
        else:
            d_act = z
            
        d_inputs = d_act @ weight.T
        self.d_w = inputs.T @ d_act
        self.d_b = d_act.sum(axis=0) 
        
        return d_inputs#, self.d_w, self.d_b
    
    def update_gd_params(self, lr):
        #print("Weight upd: ", self.d_w)
        self.weights = self.weights - lr * self.d_w
        self.biases = self.biases - lr * self.d_b

class PerceptronClassifier(BaseEstimator, ClassifierMixin, ):
    
    """
    Parameters
    ----------
    Attributes
    ----------
    Notes
    -----
    See also
    --------
    Examples
    --------
    """
    # Constructor for the classifier object
    def __init__(self, in_dim, out_dim, hidden_units, n_layers, activation = 'Sigmoid', 
                 learning_rate = 0.01, weight_decay = 0, epochs = -1, regularisation = 'L2'):

        """Setup a Perceptron classifier .
        Parameters
        ----------
        Returns
        -------
        """     
        
        self.layers = []
        self.lr = learning_rate
        self.regularisation = regularisation
        
        self.layers.append(Layer(in_dim, hidden_units, activation, 'Xavier'))
        for l in range(n_layers):
            self.layers.append(Layer(hidden_units, hidden_units, activation, 'Xavier'))
        self.layers.append(Layer(hidden_units, out_dim, activation, 'Xavier'))
        
        print("Layers:", len(self.layers))
        
        # Initialise class variabels
    def forward(self, X):
        out = self.layers[0].forward(X)
        for layer in self.layers[1:]:
            out = layer.forward(out)
        return out
                
    def backward(self, in_grad):
        i = len(self.layers) - 2 
        # d_inputs, _, _ = lay.backward(in_grad)
        next_grad = self.layers[i+1].backward(in_grad)
        while i >= 0:
            next_grad = self.layers[i].backward(next_grad)
            i -= 1
        
    def l2_loss(self, y_hat, pred):
        # totalSum = 0
        # for layer in self.layers:
        #     totalSum = totalSum + np.sum(np.sum(layer.getWeights())
        return -y_hat-np.squeeze(pred)

    def loss(self, y_hat, pred):
        return -np.expand_dims(y_hat-np.squeeze(pred),axis=1)
        
    # The fit function to train a classifier
    def fit(self, X, y, epochs = 30):
        # WRITE CODE HERE
        for i in range(epochs):
            out = self.forward(X)
            print("Prediction:",out)
            
            if (self.regularisation == 'L2'):
                grad = self.l2_loss(y, out)
            else:
                grad = self.loss(y, out)
            
            # Backpropagation
            self.backward(grad)
            
            # Update weights and biases
            for layer in self.layers:
                layer.update_gd_params(self.lr)
        return
    
    # The predict function to make a set of predictions for a set of query instances
    def predict(self, X):
        return self.forward(X)
    
    # The predict_proba function to make a set of predictions for a set of query instances. This returns a set of class distributions.
    def predict_proba(self, X):
        tmp = self.forward(X)
        sum1 = tmp.sum(axis = 1)
        out = X.T / sum1
        out = out.T
        return out

## Task 2: Evaluation

Load the Diabethic Retinopathy dataset

In [3]:
x = np.array([[1,1],[2,2]])


D = 2
N = 2
H = 1


weights = np.ones((2,1))
biases = np.ones((1))  
              


In [146]:
#(self, in_dim, out_dim, hidden_units, n_layers,
x = np.array([[0,0],[0,1],[1,0],[1,1]])
y = np.array([0,1,1,0])
clf = PerceptronClassifier(2, 1, 5, 0, regularisation='None', learning_rate = 3,activation='Sigmoid')
clf.fit(x, y,epochs=1000)

Layers: 2
Prediction: [[0.47580584]
 [0.49092422]
 [0.51666368]
 [0.52844318]]
Prediction: [[0.47113504]
 [0.48664073]
 [0.51147338]
 [0.52347949]]
Prediction: [[0.47432615]
 [0.48977983]
 [0.51425379]
 [0.52604479]]
Prediction: [[0.47265083]
 [0.48835288]
 [0.51217329]
 [0.52403435]]
Prediction: [[0.47389841]
 [0.48969511]
 [0.51308101]
 [0.52485044]]
Prediction: [[0.47332619]
 [0.4893453 ]
 [0.51220125]
 [0.52399205]]
Prediction: [[0.47380404]
 [0.49000418]
 [0.51242338]
 [0.52418199]]
Prediction: [[0.47360262]
 [0.49004793]
 [0.51200843]
 [0.523783  ]]
Prediction: [[0.47375594]
 [0.4904498 ]
 [0.51199715]
 [0.52377521]]
Prediction: [[0.47364647]
 [0.49063186]
 [0.51177002]
 [0.52357605]]
Prediction: [[0.4736382 ]
 [0.49093769]
 [0.51169401]
 [0.52353099]]
Prediction: [[0.47351468]
 [0.4911664 ]
 [0.5115533 ]
 [0.52343738]]
Prediction: [[0.47340065]
 [0.49143715]
 [0.51147473]
 [0.52341463]]
Prediction: [[0.47322108]
 [0.49168203]
 [0.51138493]
 [0.52339395]]
Prediction: [[0.47301702

Prediction: [[0.03360437]
 [0.96865529]
 [0.9581372 ]
 [0.04392765]]
Prediction: [[0.03354799]
 [0.96870983]
 [0.9582087 ]
 [0.04385116]]
Prediction: [[0.03349187]
 [0.96876412]
 [0.95827988]
 [0.04377501]]
Prediction: [[0.03343601]
 [0.96881816]
 [0.95835073]
 [0.04369921]]
Prediction: [[0.03338039]
 [0.96887195]
 [0.95842126]
 [0.04362376]]
Prediction: [[0.03332502]
 [0.96892551]
 [0.95849148]
 [0.04354866]]
Prediction: [[0.03326989]
 [0.96897882]
 [0.95856138]
 [0.04347389]]
Prediction: [[0.03321501]
 [0.96903189]
 [0.95863097]
 [0.04339947]]
Prediction: [[0.03316037]
 [0.96908472]
 [0.95870024]
 [0.04332538]]
Prediction: [[0.03310597]
 [0.96913731]
 [0.95876921]
 [0.04325162]]
Prediction: [[0.03305181]
 [0.96918967]
 [0.95883787]
 [0.0431782 ]]
Prediction: [[0.03299789]
 [0.9692418 ]
 [0.95890622]
 [0.04310511]]
Prediction: [[0.0329442 ]
 [0.96929369]
 [0.95897427]
 [0.04303234]]
Prediction: [[0.03289075]
 [0.96934536]
 [0.95904202]
 [0.0429599 ]]
Prediction: [[0.03283753]
 [0.9693

Prediction: [[0.02010708]
 [0.98162136]
 [0.97514745]
 [0.02580589]]
Prediction: [[0.02009332]
 [0.98163453]
 [0.97516469]
 [0.02578756]]
Prediction: [[0.02007959]
 [0.98164767]
 [0.9751819 ]
 [0.02576927]]
Prediction: [[0.02006588]
 [0.98166078]
 [0.97519908]
 [0.02575102]]
Prediction: [[0.0200522 ]
 [0.98167387]
 [0.97521623]
 [0.0257328 ]]
Prediction: [[0.02003854]
 [0.98168693]
 [0.97523334]
 [0.02571461]]
Prediction: [[0.02002491]
 [0.98169997]
 [0.97525042]
 [0.02569646]]
Prediction: [[0.02001131]
 [0.98171298]
 [0.97526746]
 [0.02567835]]
Prediction: [[0.01999773]
 [0.98172597]
 [0.97528448]
 [0.02566027]]
Prediction: [[0.01998418]
 [0.98173893]
 [0.97530146]
 [0.02564222]]
Prediction: [[0.01997066]
 [0.98175187]
 [0.97531841]
 [0.02562421]]
Prediction: [[0.01995716]
 [0.98176478]
 [0.97533532]
 [0.02560623]]
Prediction: [[0.01994368]
 [0.98177767]
 [0.97535221]
 [0.02558829]]
Prediction: [[0.01993023]
 [0.98179054]
 [0.97536906]
 [0.02557038]]
Prediction: [[0.01991681]
 [0.9818

In [82]:

x = np.array([[0.74552196],
 [0.76334402]])
y = np.ones(2)

In [83]:
np.expand_dims(np.squeeze(x)-y,axis=1)

array([[-0.25447804],
       [-0.23665598]])

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 2 is different from 1)

Evaluate the perfomrance of the perceptron classifier on the daibetic retinopathy dataset.

In [40]:
x = np.array([1,1])
clf.predict(x)


array([0.47069059])

In [ ]:
plt.plot(outpur)

## Task 3 & 4: Add Different Activations & Regularisation

Reimplement the PerceptronClassifier class adding an activation function option and L2 regularisation. 

In [ ]:
class PerceptronClassifier2(BaseEstimator, ClassifierMixin):
    """
    """
    # Constructor for the classifier object
    def __init__(self):

        """Setup a Perceptron classifier .
        Parameters
        ----------

        
        Returns
        -------

        """     

        # Initialise ranomd state if set
        self.random_state = random_state
        
        # Initialise class variabels

        
    # The fit function to train a classifier
    def fit(self, X, y):
        
        # WRITE CODE HERE
        

    # The predict function to make a set of predictions for a set of query instances
    def predict(self, X):

        # WRITE CODE HERE
    
    # The predict_proba function to make a set of predictions for a set of query instances. This returns a set of class distributions.
    def predict_proba(self, X):
        
        # WRITE CODE HERE

Load the dataset and explore it.

## Task 5: Reflect on the Performance of the Different Models Evaluated

Perform hyper-parameter tuning and evaluate models. 

## Test the Diabetic Retiniphaty dataset

In [ ]:
diabetic_af = pd.read_csv('messidor_features.csv', na_values='?', index_col = 0)
diabetic_af.head()
y = diabetic_af.pop('Class').values
x_raw = diabetic_af.values
print("Features: ", x_raw[0:2])
print("Class: ", y[0:10])

### Train and predict using our classifier

With a single split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_raw, y, shuffle=True, train_size = 0.7)
clf = PerceptronClassifier(len(x_train[0]), 1, )
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
accuracy = metrics.accuracy_score(y_pred, y_test)

### Grid search

Do grid search with the train set, use the test set for evaluation

In [ ]:
cv_folds = 5
param_grid ={'activation': ['Sigmoid', 'ReLu', 'TanH'], 'regularisation':['None', 'L2']}

# Perform the search
tuned_perceptron = GridSearchCV(PerceptronClassifier(), \
                            param_grid, cv=cv_folds, verbose = 2, \
                            n_jobs = -1)
cross_val_score(clf, x_train, y_train, cv=10)